In [2]:
#from sklearn.experimental import enable_iterative_imputer
#from sklearn.preprocessing import FunctionTransformer
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer #, IterativeImputer
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder, StandardScaler, MaxAbsScaler

from sklearn.metrics import precision_score, recall_score, accuracy_score, classification_report, ConfusionMatrixDisplay
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier

from sklearn import set_config
set_config(display ="diagram")

import matplotlib.pyplot as plt
%matplotlib inline

In [8]:

full_df=pd.read_csv('../data/processed/crashes.gz', compression='gzip', low_memory=False)

In [10]:
#sample_df=selected_df.sample(200000, random_state=100)
sample_df=full_df.copy()


y = sample_df['GUILTY']
x = sample_df.drop(['GUILTY'],axis=1)

X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=100, test_size=0.25, stratify=y)


In [12]:
nominal_columns = [ 'FIRST_CRASH_TYPE', 'FIRST_CONTACT_POINT', 'MANEUVER']

X_train = X_train[nominal_columns]
X_test = X_test[nominal_columns]

In [15]:
from sklearn.base import BaseEstimator, TransformerMixin

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names=attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values
    

#num_pipeline = Pipeline([
 #       ('selector', DataFrameSelector(numeric_columns)),
 #       ('imputer', SimpleImputer(strategy = 'constant', fill_value=0))
 #       ('std_scaler', StandardScaler())
 #   ])

cat_pipeline = Pipeline([
        ('selector', DataFrameSelector(nominal_columns)),
#        ('imputer', SimpleImputer(strategy = 'constant', fill_value='NA')),
        ('cat_encoder', OneHotEncoder(sparse=False, handle_unknown =  'ignore' )),
    ])


fu = FeatureUnion(transformer_list=[
        ("cat_pipeline", cat_pipeline),
    ])



In [17]:
extra_tree_pipe = Pipeline([
                          ('fu', fu),
                          ('model', ExtraTreesClassifier( max_depth=13, min_samples_leaf=3, 
                                                         min_samples_split=6, n_estimators=150 )),
                              ])


In [18]:
#pipe_grid = { 'model__min_samples_split':[4,6,8,12,24]}

pipe_grid = {}

gs_pipe = GridSearchCV(estimator=extra_tree_pipe, 
                       param_grid=pipe_grid, cv=2, scoring='roc_auc')

#gs_pipe

In [19]:
gs_pipe.fit(X_train, y_train)
gs_pipe.best_params_

{}

In [20]:
y_pred_train=gs_pipe.predict(X_train)
y_pred_test=gs_pipe.predict(X_test)

In [21]:
gs_pipe.score(X_train, y_train)

0.8460463139296776

In [22]:
print(classification_report(y_train, y_pred_train))
print('------------')
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.80      0.88      0.84    167884
           1       0.74      0.61      0.67     93447

    accuracy                           0.79    261331
   macro avg       0.77      0.75      0.76    261331
weighted avg       0.78      0.79      0.78    261331

------------
              precision    recall  f1-score   support

           0       0.80      0.88      0.84     55962
           1       0.74      0.61      0.67     31149

    accuracy                           0.78     87111
   macro avg       0.77      0.75      0.75     87111
weighted avg       0.78      0.78      0.78     87111



In [77]:
#bagged_tree_pipe.fit(X_train, y_train)
#y_pred_test = bagged_tree_pipe.predict(X_test)
#y_pred_train = bagged_tree_pipe.predict(X_train)

#print(classification_report(y_train, y_pred_train))
#print('------------')
#print(classification_report(y_test, y_pred_test))
